In [74]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Детерминированные алгоритмы
from implicit.nearest_neighbours import ItemItemRecommender, CosineRecommender, TFIDFRecommender, BM25Recommender

# Метрики
from implicit.evaluation import train_test_split
from implicit.evaluation import precision_at_k, mean_average_precision_at_k, AUC_at_k, ndcg_at_k

In [75]:
data = pd.read_csv('D:/Geekbrains/Recommendation systems/2/retail_train.csv')
data.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [77]:
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]
print(data_train.shape)
print(data_test.shape)

(2278490, 12)
(118314, 12)


### Задание 1. Weighted Random Recommendation

Напишите код для случайных рекоммендаций, в которых вероятность рекомендовать товар прямо пропорциональна логарифму продаж
- Можно сэмплировать товары случайно, но пропорционально какому-либо весу
- Например, прямопропорционально популярности. Вес = log(sales_sum товара)

In [78]:
data.head()

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.60,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.00,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.30,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.00,1631,1,0.0,0.0
4,2375,26984851472,1,8160430,1,1.50,364,-0.39,1631,1,0.0,0.0


Main steps:
1. Create 'result' df. It'll contain 'user_id', unique 'items_bought' bought by the user and 'recommended_items'.
2. Create 'weights' df: 'item_id', 'weight' 
3. 

In [198]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'items_bought']
result.head(3)

,user_id,items_bought
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."


In [199]:
popularity = data.groupby('item_id')['sales_value'].sum().reset_index()
popularity

,item_id,sales_value
0,25671,20.94
1,26081,0.99
2,26093,1.59
3,26190,1.54
4,26355,1.98
...,...,...
89046,17991689,2.49
89047,17991691,2.49
89048,18000012,19.96
89049,18024155,3.99


In [204]:
weights = np.array(popularity['sales_value'] / popularity['sales_value'].sum())
weights.sum()

1.0

In [206]:
items = np.array(popularity['item_id'])
items

array([   25671,    26081,    26093, ..., 18000012, 18024155, 18024556],
      dtype=int64)

In [208]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'items_bought']
result.head(3)

,user_id,items_bought
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107..."


### weighted random recommendation

In [207]:
def weighted_random_recommendation(items, weights, n):
    """Случайные рекоммендации
    
    Input
    -----
    items_weights: pd.DataFrame
        Датафрейм со столбцами item_id, weight. Сумма weight по всем товарам = 1
    """
    recs = np.random.choice(items, size = n, replace = False, p = weights)
    
    # Подсказка: необходимо модифицировать функцию random_recommendation()
    # your_code
    
    return recs.tolist()

In [220]:
%%time

result['weighted_random_recommendation'] = result['user_id'].apply(lambda x: weighted_random_recommendation(items, weights, n=5))
result.head()
# your_code

Wall time: 1.7 s


,user_id,items_bought,weighted_random_recommendation
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[864615, 987518, 872427, 831628, 872702]"
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[872137, 945662, 5570579, 1027372, 15777220]"
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[927277, 6039787, 6534178, 986912, 851913]"
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[1058997, 871750, 836085, 1122358, 15596805]"
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[1023385, 12263268, 878996, 6534178, 831390]"


### random recommendation

In [224]:
def random_recommendation(items, n):
    """Случайные рекоммендации"""
    
    recs = np.random.choice(items, size=n, replace=False)
    
    return recs.tolist()

In [231]:
result['random_recommendation'] = result['user_id'].apply(lambda x: random_recommendation(items, n=5))

### popularity recommendation

In [240]:
def popularity_recommendation(data, n=5):
   
    popularity.sort_values('sales_value', ascending=False, inplace=True)
    
    recs = popularity.head(n).item_id
    
    return recs.tolist()

In [242]:
result['popularity_recommendation'] = result['user_id'].apply(lambda x: popularity_recommendation(items, n=5))

### Задание 2. Расчет метрик
Рассчитайте Precision@5 для каждого алгоритма с помощью функции из вебинара 1. Какой алгоритм показывает лучшее качество?

In [256]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    bought_list = bought_list  # Тут нет [:k] !!
    recommended_list = recommended_list[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    
    return precision

In [257]:
for column_name in result.columns[1:]:
    print('%s: %f' % (column_name, result.apply(lambda row: precision_at_k(row[column_name], row['items_bought']), axis=1).mean()))

items_bought: 1.000000
weighted_random_recommendation: 0.022821
random_recommendation: 0.000979
popularity_recommendation: 0.155240


### Задание 3*. Улучшение бейзлайнов и ItemItem

- Попробуйте улучшить бейзлайны, считая их на топ-5000 товаров
- Попробуйте улучшить разные варианты ItemItemRecommender, выбирая число соседей $K$.

In [ ]:
доделать и досдать! 

In [ ]:
top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()